## 画像認識の精度を上げる手法Axrossm

In [3]:
import torch
from torch import  nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import tqdm
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt


In [4]:
# 精度を上げる手法
transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(size=(32,32) ,padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

Compose(
    RandomHorizontalFlip(p=0.5)
    RandomCrop(size=(32, 32), padding=4)
    ToTensor()
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])
)

In [5]:
# TTA
test_transform = transforms.Compose([
    transforms.FiveCrop(size=(32,32)),
    transforms.Lambda(lambda crops: torch.stack([
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])(
            transforms.ToTensor()(crop)) for crop in crops
    ])
    )
])

In [6]:
# image (バッチサイズ, クロップ数, c, h, w) = (bs, n, c, h, w)
# クラス数 = n_classes

# data準備
# train_dataset = torchvision.datasets.CIFAR10('./cifar10_data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10('./cifar10_data', train=False, download=True, transform=test_transform)
# train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, num_workers=2)



Files already downloaded and verified


In [12]:
img, label = next(iter(test_loader))
bs, n_crops, c, h, w = img.shape

In [15]:

n_classes = 10
# 簡易的なmodel
model = nn.Sequential(
    nn.Conv2d(c, 64, 3),
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(64, n_classes)
)

In [16]:

# viewもよく使われる関数です。
# １つ目の引数に-1を入れることで、２つ目の引数で指定した値にサイズ数を自動的に調整してくれます。
img = img.view(-1, c, h, w)

In [17]:
with torch.no_grad():
    out = model(img)

out = out.view(bs, n_crops, -1)
# 最終的な出力はクロップたちの平均
out = out.mean(1)
print(out.shape)

torch.Size([128, 10])
